# Preprocess the VAERS data for multi-labels

In [93]:
import os
import pandas as pd
from tqdm import tqdm
import itertools

# Load raw data

In [94]:
# load data 2020
df_vax_20 = pd.read_csv('raw/2020VAERSVAX.csv.zip', encoding='cp1252')
df_sym_20 = pd.read_csv('raw/2020VAERSSYMPTOMS.csv.zip', encoding='cp1252')
df_rpt_20 = pd.read_csv('raw/2020VAERSDATA.csv.zip', encoding='cp1252')
print('* df 20 report size:', df_rpt_20.VAERS_ID.count())
print('* df 20 symptom size:', df_sym_20.VAERS_ID.count())
print('* df 20 vax size:', df_vax_20.VAERS_ID.count())

# load data 2021
df_vax_21 = pd.read_csv('raw/2021VAERSVAX.csv.zip', encoding='cp1252')
df_sym_21 = pd.read_csv('raw/2021VAERSSYMPTOMS.csv.zip', encoding='cp1252')
df_rpt_21 = pd.read_csv('raw/2021VAERSDATA.csv.zip', encoding='cp1252')
print('* df 21 report size:', df_rpt_21.VAERS_ID.count())
print('* df 21 symptom size:', df_sym_21.VAERS_ID.count())
print('* df 21 vax size:', df_vax_21.VAERS_ID.count())

* df 20 report size: 46933
* df 20 symptom size: 57716
* df 20 vax size: 56372
* df 21 report size: 146622
* df 21 symptom size: 200255
* df 21 vax size: 149251


/home/hehuan/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Merge 2020 and 2021 data

In [95]:
df_vax = pd.concat([df_vax_20, df_vax_21])
df_sym = pd.concat([df_sym_20, df_sym_21])
df_rpt = pd.concat([df_rpt_20, df_rpt_21])

print('* df report size:', df_rpt.VAERS_ID.count())
print('* df symptom size:', df_sym.VAERS_ID.count())
print('* df vax size:', df_vax.VAERS_ID.count())

* df report size: 193555
* df symptom size: 257971
* df vax size: 205623


## Check the data

In [96]:
df_vax.head()

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,855017,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,2,SYR,LA,ZOSTER (SHINGRIX)
1,855018,UNK,UNKNOWN MANUFACTURER,NaN,1,IM,RA,VACCINE NOT SPECIFIED (OTHER)
2,855019,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,NaN,NaN,ZOSTER (SHINGRIX)
3,855020,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,IM,LA,ZOSTER (SHINGRIX)
4,855021,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,2,IM,RA,ZOSTER (SHINGRIX)


In [97]:
df_sym.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,855017,Arthralgia,22.1,Chills,22.1,Injection site pain,22.1,Pyrexia,22.1,NaN,NaN
1,855018,Chills,22.1,Fatigue,22.1,Hypertension,22.1,Hypoaesthesia,22.1,Injected limb mobility decreased,22.1
2,855018,Muscular weakness,22.1,Pain in extremity,22.1,Pyrexia,22.1,Tremor,22.1,Vertigo,22.1
3,855019,Pain,22.1,Pruritus,22.1,Rash,22.1,NaN,NaN,NaN,NaN
4,855020,Chills,22.1,Influenza like illness,22.1,Myalgia,22.1,Pain in extremity,22.1,Pyrexia,22.1


In [98]:
df_rpt.head(6).T

,0,1,2,3,4,5
VAERS_ID,855017,855018,855019,855020,855021,855022
RECVDATE,01/01/2020,01/01/2020,01/01/2020,01/01/2020,01/01/2020,01/01/2020
STATE,HI,WI,NaN,TX,NaN,FL
AGE_YRS,55,68,50,67,73,65
CAGE_YR,55,68,50,67,73,65
CAGE_MO,NaN,NaN,NaN,NaN,NaN,NaN
SEX,F,F,F,F,F,F
RPT_DATE,NaN,NaN,NaN,NaN,NaN,NaN
SYMPTOM_TEXT,Symptoms occurred almost immediately- aching j...,"Extreme pain, muscle weakness in right arm so ...",SORENESS IN THE AREA. ITCHING AND RASH,"sore arm, fever 101 , chills, and aching musc...",Patient presented to Clinic today 1/1/2020. Sh...,Expected redness/soreness roughly 24 hours aft...
DIED,NaN,NaN,NaN,NaN,NaN,NaN


## Merge the symptom labels

Now let's merge all symptom labels in one `SYMPTOMS` column

### First round, merge each row

In [99]:
def merge_symptoms(r):
    syms = []
    
    # check each symptom
    for i in [1,2,3,4,5]:
        _sym = r['SYMPTOM%s' % i]
        if pd.isna(_sym): continue
        
        # replace the blank
        _sym = _sym.replace(' ', '_')
        syms.append(_sym)
    
    return syms
    
df_sym['SYMPTOMS'] = df_sym.apply(merge_symptoms, axis=1)

df_sym.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,SYMPTOMS
0,855017,Arthralgia,22.1,Chills,22.1,Injection site pain,22.1,Pyrexia,22.1,NaN,NaN,"[Arthralgia, Chills, Injection_site_pain, Pyre..."
1,855018,Chills,22.1,Fatigue,22.1,Hypertension,22.1,Hypoaesthesia,22.1,Injected limb mobility decreased,22.1,"[Chills, Fatigue, Hypertension, Hypoaesthesia,..."
2,855018,Muscular weakness,22.1,Pain in extremity,22.1,Pyrexia,22.1,Tremor,22.1,Vertigo,22.1,"[Muscular_weakness, Pain_in_extremity, Pyrexia..."
3,855019,Pain,22.1,Pruritus,22.1,Rash,22.1,NaN,NaN,NaN,NaN,"[Pain, Pruritus, Rash]"
4,855020,Chills,22.1,Influenza like illness,22.1,Myalgia,22.1,Pain in extremity,22.1,Pyrexia,22.1,"[Chills, Influenza_like_illness, Myalgia, Pain..."


### Second round, groupby VAERS_ID

Due to the duplicated VAERS_ID in the data, we need to combine them and remove the duplicated.

In [100]:
# group the syms and reduce the duplicated ones
df_sym_grouped = df_sym.groupby(['VAERS_ID'])['SYMPTOMS'].apply(
    lambda g: list(set(list(itertools.chain.from_iterable(g.values))))
)

# convert the series to a dataframe
df_sym_grouped = df_sym_grouped.to_frame()

# set the index
df_sym_grouped.reset_index(inplace=True)

# get the number of syms
df_sym_grouped['NUM_SYMS'] = df_sym_grouped['SYMPTOMS'].apply(lambda v: len(v))

# done, now for each VAERS_ID, we have a list of symptoms
df_sym_grouped.head()

,VAERS_ID,SYMPTOMS,NUM_SYMS
0,855017,"[Chills, Arthralgia, Pyrexia, Injection_site_p...",4
1,855018,"[Chills, Fatigue, Hypoaesthesia, Muscular_weak...",10
2,855019,"[Pruritus, Pain, Rash]",3
3,855020,"[Chills, Influenza_like_illness, Pyrexia, Myal...",5
4,855021,"[Chills, Dizziness, Palpitations, Nausea]",4


## Merge the dataset

Now let's make the dataset for downstream tasks

In [101]:
# merge the report text and vaccination info
# we don't need all of the columns in each dataframe
df = df_rpt[['VAERS_ID', 'AGE_YRS', 'SEX', 'VAX_DATE', 'SYMPTOM_TEXT', 'ALLERGIES']].merge(
    df_vax[['VAERS_ID', 'VAX_TYPE', 'VAX_MANU']].drop_duplicates(subset='VAERS_ID'),
    on='VAERS_ID',
    how='left'
)

# merge the first symptoms
df = df.merge(
    df_sym_grouped[['VAERS_ID', 'SYMPTOMS', 'NUM_SYMS']].drop_duplicates(subset='VAERS_ID'),
    on='VAERS_ID',
    how='left'
)

# we only use the COVID19 vaccination data
df = df[df['VAX_TYPE']=='COVID19']

# change the default datetime format for eaiser query
df['VAX_DATE'] = pd.to_datetime(df['VAX_DATE'], format='%m/%d/%Y')

# make sure the format is string for this two
df['SYMPTOM_TEXT'] = df['SYMPTOM_TEXT'].astype(str)

# calcuate the text length for the symptoms
df['TEXT_LEN'] = df['SYMPTOM_TEXT'].apply(lambda r: len(r))

# let's see how our dataframe looks like
print('* df size:', df['VAERS_ID'].count())
df.head()

* df size: 157099


,VAERS_ID,AGE_YRS,SEX,VAX_DATE,SYMPTOM_TEXT,ALLERGIES,VAX_TYPE,VAX_MANU,SYMPTOMS,NUM_SYMS,TEXT_LEN
35000,902418,56.0,F,2020-12-15,Patient experienced mild numbness traveling fr...,none,COVID19,PFIZER\BIONTECH,"[Hypoaesthesia, Injection_site_hypoaesthesia]",2.0,110
35007,902440,35.0,F,2020-12-15,C/O Headache,NaN,COVID19,PFIZER\BIONTECH,[Headache],1.0,12
35008,902446,55.0,F,2020-12-15,"felt warm, hot and face and ears were red and ...","Contrast Dye IV contrast, shellfish, strawberry",COVID19,PFIZER\BIONTECH,"[Feeling_hot, Flushing, Erythema]",3.0,54
35017,902464,42.0,M,2020-12-15,within 15 minutes progressive light-headedness...,none,COVID19,PFIZER\BIONTECH,"[Hyperhidrosis, Dizziness, Laboratory_test_nor...",5.0,124
35018,902465,60.0,F,2020-12-15,Pt felt wave come over body @ 1218 starting in...,Biaxin,COVID19,PFIZER\BIONTECH,"[Dysgeusia, Sensory_disturbance, Parosmia, Tre...",7.0,707


# Make sample and large dataset

In [120]:
# first, we only use the data in 2021
dft = df[df['VAX_DATE']>='2021-03-01']
print('* dft size:', dft['VAERS_ID'].count())

# then, we remove those records that are too short 
dft = dft[dft['TEXT_LEN']>80]
print('* dft size:', dft['VAERS_ID'].count())

# reset the index for spliting
dft.reset_index(inplace=True)

# then drop those not used columns
dft.drop(columns=['TEXT_LEN', 'VAX_TYPE', 'index'], inplace=True)

# then drop Nan
dft.dropna(inplace=True)
print('* final dft size:', dft['VAERS_ID'].count())

dft.head()

* dft size: 31631
* dft size: 25140
* final dft size: 17097


,VAERS_ID,AGE_YRS,SEX,VAX_DATE,SYMPTOM_TEXT,ALLERGIES,VAX_MANU,SYMPTOMS,NUM_SYMS
0,1057076,68.0,F,2021-03-16,I had the expected 24-48 hours of injection si...,"Sulfa, PCN, Dilantin",MODERNA,"[Injection_site_pruritus, Injection_site_swell...",3.0
2,1062518,31.0,M,2021-03-01,"Feeling dizzy and pale, felt weak vital signs...",none known,MODERNA,"[Dizziness, Pallor, Asthenia]",3.0
3,1062652,63.0,F,2021-03-01,63 year old complained of headache and sweatin...,Penicillin,MODERNA,"[Headache, Hyperhidrosis]",2.0
4,1062673,43.0,M,2021-03-01,Pt received their second dose of the vaccine i...,hx of vasal vagal response with shots in general,PFIZER\BIONTECH,"[Dizziness, Hypotension, Pallor]",3.0
5,1062721,36.0,F,2021-03-01,Injection site it red and blotchy. Area is cu...,gluten dairy Setraline Amatriplatline Zoloft C...,PFIZER\BIONTECH,"[Rash_macular, Injection_site_erythema]",2.0


# TOP N

Let's check what are the top N symptoms

In [121]:
# Get all symptoms
sym_lists = dft.SYMPTOMS.values.tolist()

# use a dictionary as counter
counter = {}
for sym_list in sym_lists:
    for sym in sym_list:
        if sym not in counter: counter[sym] = 0
        counter[sym] += 1

# convert the counter as a dataframe for better display
df_sym_counter = pd.DataFrame(counter.items(), columns=['SYMPTOM', 'COUNT'])
df_sym_counter.sort_values(by=['COUNT'], ascending=False).head(10)

,SYMPTOM,COUNT
6,Headache,5555
23,Pyrexia,4501
45,Chills,4498
53,Fatigue,3873
54,Pain,3837
24,Nausea,3178
3,Dizziness,3083
84,Pain_in_extremity,1873
19,Dyspnoea,1586
262,Myalgia,1541


# Output top 10 

In [122]:
# we could select the top 10 symptoms according to our dataset
aes = df_sym_counter.sort_values(by=['COUNT'], ascending=False).head(10)['SYMPTOM'].tolist()
print('* top 10 symptoms:', aes)

* top 10 symptoms: ['Headache', 'Pyrexia', 'Chills', 'Fatigue', 'Pain', 'Nausea', 'Dizziness', 'Pain_in_extremity', 'Dyspnoea', 'Myalgia']


In [123]:
# sample 500 records
dft_allsample = dft.sample(n=500)

# get those are not used for sample
dft_notsample = dft[~dft.index.isin(dft_allsample.index)]

print('* dft_allsample:', dft_allsample.shape)
print('* dft_notsample:', dft_notsample.shape)

* dft_allsample: (500, 9)
* dft_notsample: (16597, 9)


In [124]:
dft_allsample.to_csv('m_sample.csv', index=False)
dft_notsample.to_csv('m_large.csv', index=False)

print('* generated m_sample.csv and m_large.csv')

* generated m_sample.csv and m_large.csv
